# CNMa
###### Correlation N Matching

In [ ]:
#import IP_address_filter
#import SG_IP_matching
#import xml_to_csv
#import virusTotal_api
import OS_API
import pandas as pd
from datetime import datetime
import time
import os
from tqdm import tqdm
import json

## Search in OpenSearch

In [ ]:
#columns of interest
COLS = ['Src IP',
        'Src Port',
        'Dst IP',
        'Dst Port',
        'commands',
        'hashes',
        'urls',
        'loggedin',
        'startTime',
        'endTime',
        'sort_num',
        'Src Country',
        'Dst Country'
       ]

SIZE = 10000

In [ ]:
def yesnocheck(param):
    while True:
        # Note: Python 2.x users should use raw_input, the equivalent of 3.x's input
        ans = input(f"{param}")
        if ans not in ['y', 'n']:
            continue            
        else:
            break
    return ans

#ans = yesnocheck("test")
#ans

In [ ]:
#query inputting

defaultquery = [
    {'range': {'startTime':{'gte': 'now-1w/w',
                            'lte': 'now',
                            'format': 'strict_date_optional_time'}}},
                 {'exists': {'field': 'hashes'}},
                 {'match': {'geoip.country_name': 'Singapore'}},
                 {'query_string': {'default_field': 'commands', 'query': 'wget OR tftp'}}
               ]


def craft_query(devmode):
    queryls = []
    #dev mode to return default eg.
    while devmode:
        return defaultquery
    
    #custom input
    #Time range
    ans = yesnocheck("Filter on Time Range? y/n")
    if ans == 'y':
        start = str(input("Input start date:"))
        end = str(input("Input end date:"))
        rnge = {"range": {"startTime": {"gte": start, "lte": end, "format": "strict_date_optional_time"}}}
        queryls.append(rnge)    

    #Existing field
    while True:
        ans = yesnocheck("Filter if a field Exists? y/n")
        #print(ans)
        if ans == 'y':
            exist = input("Please enter field:")
            exst = {'exists': {'field': exist}}
            queryls.append(exst)
        else:
            break
    
    #Exact match on field
    while True:
        ans = yesnocheck("Exact match on a field? y/n")
        #print(ans)
        if ans == 'y':
            field = input("Please enter field:")
            value = input("Enter value to match:")
            mtch = {"match": {field: value}}
            queryls.append(mtch)
        else:
            break
    
    #General match query
    while True:
        ans = yesnocheck("General search query on a field? y/n")
        #print(ans)
        if ans == 'y':
            field = input("Please enter field:")
            query = input("Enter query:")
            qry = {"query_string": {"default_field": field, "query": query}}
            queryls.append(qry)
        else:
            break
    
    return queryls
    
#queryls = craft_query(True)
#queryls

In [ ]:
size = 10000
search_after = [0]
df_os = pd.DataFrame(columns=COLS)

queryls = craft_query(True)
print(queryls)

total = 0

while (size == SIZE):
    print(f"Continue from record: {search_after}")
    data = OS_API.opensearch_request(size,queryls,search_after)
    df_os = pd.concat([df_os,data])
    
    #update params
    size = len(data) #exit once it completes last batch
    total+=size #update total records number
    search_after = data['sort_num'][len(data)-1] #update sort number to continue after
    #time.sleep(1)
    
print(f'Done. Total records: {total}')

In [ ]:
df_os

In [ ]:
df_os[df_os['Dst Country'].isna()].sort_values(by=['startTime'])

In [ ]:
search_after = [0]
data = OS_API.opensearch_request(queryls = queryls , search_after = search_after)
data

In [ ]:
'''
#export to csv
df_os.to_csv("opensearch_matched" + "_" + str(datetime.now().strftime('%Y_%m_%d_%H_%M_%S')) + ".csv", index=False)
'''

In [ ]:
#Date inputs
year = "2022"
print('start month?')
m1 = input()
print('start day?')
d1 = input()
print('end month?')
m2 = input()
print('end day?')
d2 = input()

start_date = f"{year}-{str(100+int(m1))[-2:]}-{str(100+int(d1))[-2:]}T00:00:00.000Z"
end_date = f"{year}-{str(100+int(m2))[-2:]}-{str(100+int(d2))[-2:]}T00:00:00.000Z"
print(f"Start date: {start_date}\n End date: {end_date}")